In [1]:
import numpy as np
import cv2
import torch
from torchvision import transforms
from collections import deque

# Fungsi tambahan (pastikan fungsi-fungsi ini didefinisikan atau diimpor)
from models.experimental import attempt_load
from utils.general import non_max_suppression_kpt
from utils.datasets import letterbox
from utils.plots import output_to_keypoint, plot_skeleton_kpts

# Inisialisasi model YOLOv7
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = attempt_load('yolov7-w6-pose.pt', map_location=device)
_ = model.float().eval()
if torch.cuda.is_available():
    model.half().to(device)

class FallDetector:
    def __init__(self, fps=30):
        self.LEFT_SHOULDER, self.RIGHT_SHOULDER = 5, 6
        self.LEFT_HIP, self.RIGHT_HIP = 11, 12
        self.LEFT_ANKLE, self.RIGHT_ANKLE = 13, 14
        
        self.ALPHA = 0.4  # Diperbesar untuk toleransi lebih
        self.ANGLE_THRESHOLD = 120  # Diperbarui untuk mendeteksi sudut lebih besar
        self.VELOCITY_THRESHOLD = 0.5  # Diperbesar
        self.CONFIRMATION_FRAMES = 10  # Diperpanjang untuk mengurangi false positive
        
        self.previous_positions = deque(maxlen=5)
        self.fall_frames = 0
        self.is_fallen = False
        self.fps = fps

    def calculate_length_factor(self, shoulder, hip):
        return np.sqrt((shoulder[0] - hip[0])**2 + (shoulder[1] - hip[1])**2)

    def calculate_body_dimensions(self, keypoints):
        ls = keypoints[self.LEFT_SHOULDER*3:(self.LEFT_SHOULDER+1)*3]
        rs = keypoints[self.RIGHT_SHOULDER*3:(self.RIGHT_SHOULDER+1)*3]
        la = keypoints[self.LEFT_ANKLE*3:(self.LEFT_ANKLE+1)*3]
        
        if ls[2] > 0.5 and rs[2] > 0.5 and la[2] > 0.5:
            body_height = abs(((ls[1] + rs[1])/2) - la[1])
            body_width = abs(ls[0] - rs[0])
            return body_height, body_width
        return None, None

    def calculate_velocity(self, current_pos):
        if len(self.previous_positions) >= 2:
            prev_pos = self.previous_positions[-1]
            dx = current_pos[0] - prev_pos[0]
            dy = current_pos[1] - prev_pos[1]
            distance = np.sqrt(dx**2 + dy**2)
            time_elapsed = 1 / self.fps  # Waktu antara frame
            return distance / time_elapsed  # Kecepatan dalam pixel/detik
        return 0

    def calculate_torso_leg_angle(self, keypoints):
        ls = keypoints[self.LEFT_SHOULDER*3:(self.LEFT_SHOULDER+1)*3]
        lh = keypoints[self.LEFT_HIP*3:(self.LEFT_HIP+1)*3]
        la = keypoints[self.LEFT_ANKLE*3:(self.LEFT_ANKLE+1)*3]
        
        if all(kp[2] > 0.5 for kp in [ls, lh, la]):
            torso_vec = np.array([lh[0] - ls[0], lh[1] - ls[1]])
            leg_vec = np.array([la[0] - lh[0], la[1] - lh[1]])
            
            # Hindari vektor nol
            if np.linalg.norm(torso_vec) < 1e-6 or np.linalg.norm(leg_vec) < 1e-6:
                return None
                
            unit_torso = torso_vec / np.linalg.norm(torso_vec)
            unit_leg = leg_vec / np.linalg.norm(leg_vec)
            angle = np.degrees(np.arccos(np.clip(np.dot(unit_torso, unit_leg), -1.0, 1.0)))
            return angle
        return None

    def detect_fall(self, keypoints):
        ls = keypoints[self.LEFT_SHOULDER*3:(self.LEFT_SHOULDER+1)*3]
        rs = keypoints[self.RIGHT_SHOULDER*3:(self.RIGHT_SHOULDER+1)*3]
        lh = keypoints[self.LEFT_HIP*3:(self.LEFT_HIP+1)*3]
        rh = keypoints[self.RIGHT_HIP*3:(self.RIGHT_HIP+1)*3]
        la = keypoints[self.LEFT_ANKLE*3:(self.LEFT_ANKLE+1)*3]
        ra = keypoints[self.RIGHT_ANKLE*3:(self.RIGHT_ANKLE+1)*3]
        
        # Validasi keypoints
        if any(kp[2] < 0.5 for kp in [ls, rs, lh, rh, la, ra]):
            self.fall_frames = max(0, self.fall_frames - 1)
            return False
        
        shoulder_center = ((ls[0] + rs[0])/2, (ls[1] + rs[1])/2)
        self.previous_positions.append(shoulder_center)
        
        length_factor = self.calculate_length_factor(ls, lh)
        body_height, body_width = self.calculate_body_dimensions(keypoints)
        
        # Kondisi ketinggian yang diperbaiki
        vertical_diff_left = ls[1] - la[1]
        vertical_diff_right = rs[1] - ra[1]
        height_condition = (vertical_diff_left > self.ALPHA * length_factor) or \
                          (vertical_diff_right > self.ALPHA * length_factor)
        
        # Kondisi dimensi lebih toleran
        dimension_condition = (body_height is not None) and (body_height < body_width * 1.2)
        
        velocity = self.calculate_velocity(shoulder_center)
        angle = self.calculate_torso_leg_angle(keypoints)
        
        is_fall = False
        if height_condition and dimension_condition:
            angle_condition = (angle is not None) and (angle < self.ANGLE_THRESHOLD)
            velocity_condition = velocity > self.VELOCITY_THRESHOLD
            is_fall = angle_condition or velocity_condition  # Menggunakan OR logic
        
        if is_fall:
            self.fall_frames += 1
            if self.fall_frames >= self.CONFIRMATION_FRAMES:
                self.is_fallen = True
                return True
        else:
            self.fall_frames = max(0, self.fall_frames - 2)
            if self.fall_frames == 0:
                self.is_fallen = False
        
        return self.is_fallen

def process_video(video_path, output_path=None, rotation=0):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return
    
    # Mendapatkan properti video
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Menyesuaikan dimensi untuk rotasi
    if rotation in [90, 270]:
        width, height = original_height, original_width
    else:
        width, height = original_width, original_height
    
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    fall_detector = FallDetector(fps=fps)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Rotasi frame
        if rotation == 90:
            frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        elif rotation == 180:
            frame = cv2.rotate(frame, cv2.ROTATE_180)
        elif rotation == 270:
            frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
        
        # Preprocessing dengan letterbox
        # Ganti baris letterbox:
        img, (pad_w, pad_h), (ratio_w, ratio_h) = letterbox(frame, new_shape=640, stride=64, auto=True)
        img_tensor = transforms.ToTensor()(img)
        img_tensor = img_tensor.unsqueeze(0).to(device)
        if torch.cuda.is_available():
            img_tensor = img_tensor.half()
        
        # Inference
        with torch.no_grad():
            output, _ = model(img_tensor)
            output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
            output = output_to_keypoint(output)
        
        # Menyiapkan frame output
        display_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Memproses setiap orang yang terdeteksi
        if output is not None and len(output) > 0:
            for idx in range(output.shape[0]):
                keypoints = output[idx, 7:].T
                
                # Menyesuaikan keypoints ke koordinat asli
                keypoints[0::3] = (keypoints[0::3] - pad_w) / ratio_w  # X
                keypoints[1::3] = (keypoints[1::3] - pad_h) / ratio_h  # Y
                
                # Plot skeleton
                plot_skeleton_kpts(display_img, keypoints, 3)
                
                # Deteksi jatuh
                if fall_detector.detect_fall(keypoints):
                    cv2.putText(display_img, "FALL DETECTED!", (50, 80), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)
                    
                    # Menampilkan metrik
                    body_height, body_width = fall_detector.calculate_body_dimensions(keypoints)
                    angle = fall_detector.calculate_torso_leg_angle(keypoints)
                    
                    cv2.putText(display_img, f"H/W Ratio: {body_height/body_width:.2f}", (50, 120),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
                    cv2.putText(display_img, f"Angle: {angle:.1f}°" if angle else "Angle: N/A", (50, 150),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        
        # Konversi ke BGR untuk output
        display_img = cv2.cvtColor(display_img, cv2.COLOR_RGB2BGR)
        
        # Menampilkan dan menyimpan
        cv2.imshow("Fall Detection", display_img)
        if output_path:
            out.write(display_img)
            
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()

# Contoh penggunaan
process_video('C:/Users/LENOVO/Documents/A Skripsi/datasets/FallDataset/Dataset/Coffee_room_01/Videos/video (1).avi', None)

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL models.yolo.Model was not an allowed global by default. Please use `torch.serialization.add_safe_globals([Model])` or the `torch.serialization.safe_globals([Model])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default with weights_only https

SyntaxError: trailing comma not allowed without surrounding parentheses (741100123.py, line 9)